## RC4Space/OrcaTech Machine Learning Presentation
## 17 September 2020
## Ong Ming Lun

#### Made for the OrcaTech presentation, contact @mung_lin on telegram for more details. Please ping me if you want to highlight any issues or use the code for your own purposes too. Thank you.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### 1. Import the MNIST Dataset

In [ ]:
from keras.datasets import mnist
(Xtrain, Ytrain), (Xtest, Ytest) = mnist.load_data()

In [ ]:
print(Xtrain.shape)
print("Training Set: %s images of %s x %s pixels each" %(Xtrain.shape[0],Xtrain.shape[1],Xtrain.shape[2]))

In [ ]:
print(Xtest.shape)
print("Training Set: %s images of %s x %s pixels each" %(Xtest.shape[0],Xtest.shape[1],Xtest.shape[2]))

#### Attempt to display some images in the dataset

#### Training Set

In [ ]:
for i in range(4):
    plt.subplot(221 + i)
    plt.imshow(Xtrain[i], cmap=plt.get_cmap('gray'))
    plt.axis('off')
    plt.title("Ground Truth: " + str(Ytrain[i]) )
plt.show()

#### Test Set

In [ ]:
for i in range(4):
    plt.subplot(221 + i)
    plt.imshow(Xtest[i], cmap=plt.get_cmap('gray'))
    plt.axis('off')
    plt.title("Ground Truth: " + str(Ytest[i]) )
plt.show()    

#### Preprocessing

In [ ]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score, KFold

In [ ]:
Xtrain_reshape = Xtrain.reshape(Xtrain.shape[0],-1)
Xtest_reshape = Xtest.reshape(Xtest.shape[0],-1)

scaler = preprocessing.MinMaxScaler()
scaler.fit(Xtrain_reshape)
Xtrain_reshape = scaler.transform(Xtrain_reshape)
Xtest_reshape = scaler.transform(Xtest_reshape)

### A. Building different models for the same problem

### <font color=red>Takeaway 1 <br> <br> Using different models for the same problem: Logistic Regression, Random Forest, MLP, CNNs</font>

#### 1. Logistic Regression (Classifier)

In [ ]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(verbose=2)
LR.fit(Xtrain_reshape[:10000,:],Ytrain[:10000])
#LR.fit(Xtrain_reshape,Ytrain)
print("Training Accuracy: " + str(accuracy_score(LR.predict(Xtrain_reshape[:10000,:]),Ytrain[:10000])))
print("Test Accuracy: " + str(accuracy_score(LR.predict(Xtest_reshape),Ytest)))

#### 2. Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
RandForest = RandomForestClassifier(n_estimators=20,max_depth=8,min_samples_leaf=2)
RandForest.fit(Xtrain_reshape[:1000,:],Ytrain[:1000])
print("Training Accuracy: " + str(accuracy_score(RandForest.predict(Xtrain_reshape[:1000,:]),Ytrain[:1000])))
print("Test Accuracy: " + str(accuracy_score(RandForest.predict(Xtest_reshape),Ytest)))

#### <font color=red>Takeaway 2 <br> <br> Amount of Training Input Data <br> <br> More data gets you more accurate results.</font>

In [ ]:
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier(n_estimators=200,max_depth=25,min_samples_leaf=7)
RF.fit(Xtrain_reshape[:10000,:],Ytrain[:10000])
print("Training Accuracy: " + str(accuracy_score(RF.predict(Xtrain_reshape[:10000,:]),Ytrain[:10000])))
print("Test Accuracy: " + str(accuracy_score(RF.predict(Xtest_reshape),Ytest)))

#### 3. Multilayer Perceptron Classifier

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout
NN = keras.Sequential()
NN.add(Dense(50, activation='relu'))
NN.add(Dropout(0.7))
NN.add(Dense(10, activation='softmax'))

NN.compile(loss='sparse_categorical_crossentropy',optimizer=tf.keras.optimizers.Adam(0.0002),metrics=['accuracy'])
NN.fit(Xtrain_reshape[:10000,:],Ytrain[:10000],batch_size=200,epochs=100)

In [ ]:
print("NN")
print("Training Accuracy: " + str(accuracy_score(NN.predict_classes(Xtrain_reshape[:10000,:]),Ytrain[:10000])))
print("Test Accuracy: " + str(accuracy_score(NN.predict_classes(Xtrain_reshape[:1000,:]),Ytrain[:1000])))

#### <font color=red>Takeaway 3 <br> <br> Do not mix training and test data!!!</font>

In [ ]:
NN_deep = keras.Sequential()
NN_deep.add(Dense(50, activation='relu'))
NN_deep.add(Dense(30, activation='relu'))
NN_deep.add(Dense(10, activation='softmax'))

NN_deep.compile(loss='sparse_categorical_crossentropy',optimizer=tf.keras.optimizers.Adam(0.002),metrics=['accuracy'])
NN_deep.fit(Xtrain_reshape,Ytrain,batch_size=200,epochs=100)

In [ ]:
print("NN")
print("Training Accuracy: " + str(accuracy_score(NN_deep.predict_classes(Xtrain_reshape[:10000,:]),Ytrain[:10000])))
print("Test Accuracy: " + str(accuracy_score(NN_deep.predict_classes(Xtrain_reshape[:1000,:]),Ytrain[:1000])))

#### <font color=red>Takeaway 4 <br> <br> If you're not careful... your model can overfit</font>

#### <font color=red>A few things can be done to prevent overfitting: hyperparameter optimization, reducing the learning_rate, reducing the network size/parameters used, dropout </font>

#### <font color=red>For this instance, we attempt dropout to randomly set the nodes in the network to zero.</font>

In [ ]:
MLP = keras.Sequential()
MLP.add(Dense(50, activation='relu'))
MLP.add(Dropout(0.7))
MLP.add(Dense(30, activation='relu'))
MLP.add(Dropout(0.7))
MLP.add(Dense(10, activation='softmax'))

MLP.compile(loss='sparse_categorical_crossentropy',optimizer=tf.keras.optimizers.Adam(0.0002),metrics=['accuracy'])
MLP.fit(Xtrain_reshape,Ytrain,batch_size=200,epochs=100)

In [ ]:
print("MLP")
print("Training Accuracy: " + str(accuracy_score(MLP.predict_classes(Xtrain_reshape[:10000,:]),Ytrain[:10000])))
print("Test Accuracy: " + str(accuracy_score(MLP.predict_classes(Xtest_reshape),Ytest)))

In [ ]:
MLP.summary()

#### 4. Convolutional Neural Network Classifier

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten
CNN = keras.Sequential()
CNN.add(Conv2D(12, (3, 3), activation='relu', input_shape=(Xtrain.shape[1], Xtrain.shape[2], 1)))
CNN.add(MaxPooling2D((2, 2)))
CNN.add(Conv2D(16, (4, 4), activation='relu'))
CNN.add(MaxPooling2D((2, 2)))
CNN.add(Conv2D(16, (3, 3), activation='relu'))
CNN.add(Flatten())
CNN.add(Dense(10, activation='softmax'))
CNN.compile(loss='sparse_categorical_crossentropy',optimizer=tf.keras.optimizers.Adam(0.0002),metrics=['accuracy'])
CNN.fit(Xtrain_reshape.reshape((Xtrain.shape[0], Xtrain.shape[1], Xtrain.shape[1], 1)),Ytrain,batch_size=100,epochs=30)

In [ ]:
print("CNN")
print("Training Accuracy: " + str(accuracy_score(CNN.predict_classes( Xtrain_reshape.reshape(Xtrain.shape[0], Xtrain.shape[1], Xtrain.shape[1], 1)  ),Ytrain)))
print("Test Accuracy: " + str(accuracy_score(CNN.predict_classes( Xtest_reshape.reshape(Xtest.shape[0], Xtest.shape[1], Xtest.shape[1], 1)  ),Ytest)))

In [ ]:
CNN.summary()

In [ ]:
CNN.save('CNN.h5')
#CNN_load = tf.keras.models.load_model('CNN.h5')
#print("Training Accuracy: " + str(accuracy_score(CNN_load.predict_classes( Xtrain_reshape.reshape(Xtrain.shape[0], Xtrain.shape[1], Xtrain.shape[1], 1)  ),Ytrain)))
#print("Test Accuracy: " + str(accuracy_score(CNN_load.predict_classes( Xtest_reshape.reshape(Xtest.shape[0], Xtest.shape[1], Xtest.shape[1], 1)  ),Ytest)))

### Machine Learning Concepts

#### <font color=red> 5. Visualising some of the incorrect predictions </font>

In [ ]:
wrong_indices = np.where(np.not_equal(MLP.predict_classes(Xtest_reshape), Ytest))

In [ ]:
Ytest[wrong_indices]

In [ ]:
MLP.predict_classes(Xtest_reshape)[wrong_indices]

In [ ]:
for i in range(4):
    plt.subplot(221 + i)
    plt.imshow(Xtest[wrong_indices[0][i]], cmap=plt.get_cmap('gray'))
    plt.axis('off')
    plt.title("Truth: " + str(Ytest[wrong_indices[0][i]]) + ", Predicted: " +str(MLP.predict_classes(Xtest_reshape)[wrong_indices[0][i]])  )
plt.show()

In [ ]:
for i in range(4):
    plt.subplot(221 + i)
    plt.imshow(Xtest[wrong_indices[0][4+i]], cmap=plt.get_cmap('gray'))
    plt.axis('off')
    plt.title("Truth: " + str(Ytest[wrong_indices[0][i+4]]) + ", Predicted: " +str(MLP.predict_classes(Xtest_reshape)[wrong_indices[0][4+i]])  )
plt.show()

#### <font color=red> Neural network parameters: Optimisers, number of epoches, network parameters, optimisers, batch size, different types of layers, convolutional layers VS dense layers </font>

#### <font color=red> Machine Learning concepts: Backpropagation, global vs local minima, explainability, hyperparameter optimization, activation functions, probability outputs </font>

#### For further reference: Stanford CS231n Convolutional Neural Networks, 2 Blue 1 Brown Neural Networks, Tensorflow Documentation